In [3]:
import os, cv2, numpy as np
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetV2B3
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# ✅ STEP 1: Paths - Updated for Corn crop only
BASE = "/kaggle/input/team-11a-crop-wise-dataset/Dataset_annam"
RAW_TRAIN = os.path.join("/kaggle/input/team-11a-crop-wise-dataset/Dataset_annam/Corn/Train")  # Only Corn training data
RAW_VAL = os.path.join("/kaggle/input/team-11a-crop-wise-dataset/Dataset_annam/Corn/Valid")    # Only Corn validation data
PREP_TRAIN = "/kaggle/working/train_preprocessed_corn"
PREP_VAL = "/kaggle/working/val_preprocessed_corn"
IMG_SIZE = (256, 256)

# ✅ STEP 2: Enhance with CLAHE (modified for Corn subclasses)
def enhance_images(src, dst):
    if not os.path.exists(dst): os.makedirs(dst)
    for disease_class in os.listdir(src):
        src_class = os.path.join(src, disease_class)
        dst_class = os.path.join(dst, disease_class)
        os.makedirs(dst_class, exist_ok=True)
        for img in tqdm(os.listdir(src_class), desc=f"Processing {disease_class}"):
            try:
                path = os.path.join(src_class, img)
                image = cv2.imread(path)
                if image is None:
                    continue
                image = cv2.resize(image, IMG_SIZE)
                lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
                l, a, b = cv2.split(lab)
                clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
                cl = clahe.apply(l)
                limg = cv2.merge((cl, a, b))
                final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
                cv2.imwrite(os.path.join(dst_class, img), final)
            except Exception as e:
                print(f"Error processing {path}: {str(e)}")
                continue

# ✅ Only run once
enhance_images(RAW_TRAIN, PREP_TRAIN)
enhance_images(RAW_VAL, PREP_VAL)

Processing Corn_Cercospora_leaf_spot Gray_leaf_spot: 100%|██████████| 2216/2216 [00:18<00:00, 120.35it/s]
Processing Corn_(maize)healthy: 100%|██████████| 1859/1859 [00:13<00:00, 139.01it/s]
Processing Corn_Cercospora_leaf_spot Gray_leaf_spot: 100%|██████████| 430/430 [00:03<00:00, 135.64it/s]
Processing Corn_(maize)healthy: 100%|██████████| 465/465 [00:02<00:00, 161.23it/s]


In [4]:
# ✅ STEP 3: Class Check - For Corn disease classes
assert sorted(os.listdir(PREP_TRAIN)) == sorted(os.listdir(PREP_VAL)), "Train/Val class mismatch in Corn data"
NUM_CLASSES = len(os.listdir(PREP_TRAIN))
print(f"Number of Corn disease classes: {NUM_CLASSES}")

# ✅ STEP 4: Augmentation
train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    zoom_range=0.25,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    brightness_range=(0.8, 1.2),
    channel_shift_range=20.0,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

val_gen = ImageDataGenerator(rescale=1./255)

train_data = train_gen.flow_from_directory(
    PREP_TRAIN, 
    target_size=IMG_SIZE, 
    batch_size=32, 
    class_mode='categorical', 
    shuffle=True,
    classes=sorted(os.listdir(PREP_TRAIN))  # Ensure consistent class ordering
)

val_data = val_gen.flow_from_directory(
    PREP_VAL, 
    target_size=IMG_SIZE, 
    batch_size=32, 
    class_mode='categorical', 
    shuffle=False,
    classes=sorted(os.listdir(PREP_TRAIN))  # Same classes as training
)

# ✅ STEP 5: Build Model (Keras built-in)
base_model = EfficientNetV2B3(include_top=False, weights='imagenet', input_shape=(256, 256, 3))
x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.4)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(NUM_CLASSES, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=output)

# ✅ STEP 6: Compile
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=['accuracy']
)

# ✅ STEP 7: Callbacks
callbacks = [
    ModelCheckpoint("best_corn_model.h5", save_best_only=True, monitor='val_accuracy'),
    ReduceLROnPlateau(monitor='val_loss', patience=2, factor=0.3, verbose=1),
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
]

# ✅ STEP 8: Phase 1 - Train Head
base_model.trainable = False
print("✅ Phase 1: Training classifier head for Corn diseases...")
model.fit(train_data, validation_data=val_data, epochs=8, callbacks=callbacks)

Number of Corn disease classes: 4
Found 9196 images belonging to 4 classes.
Found 1867 images belonging to 4 classes.


I0000 00:00:1751569072.880982      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1751569072.881615      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


52606240/52606240 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
✅ Phase 1: Training classifier head for Corn diseases...


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/8


I0000 00:00:1751569104.828591     114 service.cc:148] XLA service 0x7f53d0002370 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1751569104.829425     114 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1751569104.829445     114 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1751569107.872362     114 cuda_dnn.cc:529] Loaded cuDNN version 90300


  1/288 ━━━━━━━━━━━━━━━━━━━━ 3:49:29 48s/step - accuracy: 0.1875 - loss: 1.4436

I0000 00:00:1751569125.633483     114 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


288/288 ━━━━━━━━━━━━━━━━━━━━ 235s 651ms/step - accuracy: 0.3142 - loss: 1.4089 - val_accuracy: 0.2651 - val_loss: 1.4048 - learning_rate: 0.0010
Epoch 2/8
288/288 ━━━━━━━━━━━━━━━━━━━━ 155s 537ms/step - accuracy: 0.3278 - loss: 1.3711 - val_accuracy: 0.2651 - val_loss: 1.3846 - learning_rate: 0.0010
Epoch 3/8
288/288 ━━━━━━━━━━━━━━━━━━━━ 155s 539ms/step - accuracy: 0.3539 - loss: 1.3642 - val_accuracy: 0.2732 - val_loss: 1.3842 - learning_rate: 0.0010
Epoch 4/8
288/288 ━━━━━━━━━━━━━━━━━━━━ 155s 539ms/step - accuracy: 0.3514 - loss: 1.3608 - val_accuracy: 0.2753 - val_loss: 1.3674 - learning_rate: 0.0010
Epoch 5/8
288/288 ━━━━━━━━━━━━━━━━━━━━ 154s 534ms/step - accuracy: 0.3608 - loss: 1.3556 - val_accuracy: 0.2694 - val_loss: 1.3737 - learning_rate: 0.0010
Epoch 6/8
288/288 ━━━━━━━━━━━━━━━━━━━━ 0s 527ms/step - accuracy: 0.3683 - loss: 1.3512
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0003000000142492354.
288/288 ━━━━━━━━━━━━━━━━━━━━ 157s 545ms/step - accuracy: 0.3683 - loss: 

In [5]:
# ✅ STEP 9: Phase 2 - Fine-tune All
base_model.trainable = True
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=['accuracy']
)
print("✅ Phase 2: Fine-tuning full model for Corn diseases...")
history = model.fit(train_data, validation_data=val_data, epochs=20, callbacks=callbacks)

# ✅ STEP 10: Evaluation
val_loss, val_acc = model.evaluate(val_data)
print(f"\n✅ Final Validation Accuracy for Corn diseases: {val_acc:.4f}")
print(f"🔻 Final Validation Loss for Corn diseases: {val_loss:.4f}")

# Print class indices for reference
print("\nClass Indices:", train_data.class_indices)

✅ Phase 2: Fine-tuning full model for Corn diseases...
Epoch 1/20


E0000 00:00:1751570570.601201     115 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1751570570.747361     115 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1751570571.276455     115 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1751570571.428488     115 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1751570571.917060     115 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:0

  4/288 ━━━━━━━━━━━━━━━━━━━━ 2:16 480ms/step - accuracy: 0.1914 - loss: 1.6583

E0000 00:00:1751570623.950587     115 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1751570624.088733     115 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1751570624.466131     115 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1751570624.607511     115 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


288/288 ━━━━━━━━━━━━━━━━━━━━ 386s 828ms/step - accuracy: 0.4054 - loss: 1.3380 - val_accuracy: 0.7327 - val_loss: 0.8812 - learning_rate: 1.0000e-05
Epoch 2/20
288/288 ━━━━━━━━━━━━━━━━━━━━ 167s 578ms/step - accuracy: 0.7807 - loss: 0.8128 - val_accuracy: 0.8848 - val_loss: 0.6023 - learning_rate: 1.0000e-05
Epoch 3/20
288/288 ━━━━━━━━━━━━━━━━━━━━ 168s 581ms/step - accuracy: 0.8653 - loss: 0.6452 - val_accuracy: 0.9239 - val_loss: 0.5229 - learning_rate: 1.0000e-05
Epoch 4/20
288/288 ━━━━━━━━━━━━━━━━━━━━ 167s 577ms/step - accuracy: 0.9026 - loss: 0.5883 - val_accuracy: 0.9389 - val_loss: 0.4983 - learning_rate: 1.0000e-05
Epoch 5/20
288/288 ━━━━━━━━━━━━━━━━━━━━ 167s 579ms/step - accuracy: 0.9090 - loss: 0.5654 - val_accuracy: 0.9432 - val_loss: 0.4829 - learning_rate: 1.0000e-05
Epoch 6/20
288/288 ━━━━━━━━━━━━━━━━━━━━ 167s 578ms/step - accuracy: 0.9206 - loss: 0.5450 - val_accuracy: 0.9464 - val_loss: 0.4739 - learning_rate: 1.0000e-05
Epoch 7/20
288/288 ━━━━━━━━━━━━━━━━━━━━ 167s 577ms/

In [ ]:
import os, cv2, numpy as np
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetV2B3
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# ✅ Configuration
BASE = "/kaggle/input/team-11a-crop-wise-dataset/Dataset_annam"
IMG_SIZE = (256, 256)
CROPS = ['Corn', 'Cotton', 'Grape', 'Potato', 'Rice', 'Sugarcane', 'Tomato', 'Wheat']  # Note: 'Cotton' was misspelled as 'Cotton' in your original structure

# First, let's verify the actual dataset structure
print("Available directories in dataset root:")
print(os.listdir(BASE))

# ✅ Function to train model for a single crop
def train_crop_model(crop_name):
    print(f"\n{'='*50}")
    print(f"🚀 Starting training for {crop_name} crop")
    print(f"{'='*50}\n")
    
    # Updated paths based on your dataset structure
    raw_train = os.path.join(BASE, crop_name, "Train")
    raw_val = os.path.join(BASE, crop_name, "Valid")
    
    # Verify paths exist
    if not os.path.exists(raw_train):
        print(f"❌ Training directory not found: {raw_train}")
        return None
    if not os.path.exists(raw_val):
        print(f"❌ Validation directory not found: {raw_val}")
        return None
    
    prep_train = f"/kaggle/working/train_preprocessed_{crop_name.lower()}"
    prep_val = f"/kaggle/working/val_preprocessed_{crop_name.lower()}"
    
    # ✅ Preprocessing
    def enhance_images(src, dst):
        if not os.path.exists(dst): os.makedirs(dst)
        for disease_class in os.listdir(src):
            src_class = os.path.join(src, disease_class)
            dst_class = os.path.join(dst, disease_class)
            os.makedirs(dst_class, exist_ok=True)
            for img in tqdm(os.listdir(src_class), desc=f"Processing {disease_class}"):
                try:
                    path = os.path.join(src_class, img)
                    image = cv2.imread(path)
                    if image is None:
                        continue
                    image = cv2.resize(image, IMG_SIZE)
                    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
                    l, a, b = cv2.split(lab)
                    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
                    cl = clahe.apply(l)
                    limg = cv2.merge((cl, a, b))
                    final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
                    cv2.imwrite(os.path.join(dst_class, img), final)
                except Exception as e:
                    print(f"Error processing {path}: {str(e)}")
                    continue
    
    # Run preprocessing
    print("Preprocessing training data...")
    enhance_images(raw_train, prep_train)
    print("Preprocessing validation data...")
    enhance_images(raw_val, prep_val)
    
    # Verify classes
    try:
        assert sorted(os.listdir(prep_train)) == sorted(os.listdir(prep_val)), f"Train/Val class mismatch in {crop_name} data"
        num_classes = len(os.listdir(prep_train))
        print(f"Number of {crop_name} disease classes: {num_classes}")
    except Exception as e:
        print(f"❌ Error verifying classes: {str(e)}")
        return None
    
    # Data generators
    train_gen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=30,
        zoom_range=0.25,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.15,
        brightness_range=(0.8, 1.2),
        channel_shift_range=20.0,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest'
    )
    
    val_gen = ImageDataGenerator(rescale=1./255)
    
    train_data = train_gen.flow_from_directory(
        prep_train,
        target_size=IMG_SIZE,
        batch_size=32,
        class_mode='categorical',
        shuffle=True,
        classes=sorted(os.listdir(prep_train))
    )
    
    val_data = val_gen.flow_from_directory(
        prep_val,
        target_size=IMG_SIZE,
        batch_size=32,
        class_mode='categorical',
        shuffle=False,
        classes=sorted(os.listdir(prep_train))
    )
    
    # Model architecture
    base_model = EfficientNetV2B3(include_top=False, weights='imagenet', input_shape=(256, 256, 3))
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.4)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.3)(x)
    output = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=output)
    
    # Compile
    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-3),
        loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
        metrics=['accuracy']
    )
    
    # Callbacks
    callbacks = [
        ModelCheckpoint(f"best_{crop_name.lower()}_model.h5", save_best_only=True, monitor='val_accuracy'),
        ReduceLROnPlateau(monitor='val_loss', patience=2, factor=0.3, verbose=1),
        EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    ]
    
    # Training
    print(f"\n✅ Phase 1: Training classifier head for {crop_name} diseases...")
    base_model.trainable = False
    model.fit(train_data, validation_data=val_data, epochs=8, callbacks=callbacks)
    
    print(f"\n✅ Phase 2: Fine-tuning full model for {crop_name} diseases...")
    base_model.trainable = True
    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-5),
        loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
        metrics=['accuracy']
    )
    history = model.fit(train_data, validation_data=val_data, epochs=20, callbacks=callbacks)
    
    # Evaluation
    val_loss, val_acc = model.evaluate(val_data)
    print(f"\n✅ Final Validation Accuracy for {crop_name}: {val_acc:.4f}")
    print(f"🔻 Final Validation Loss for {crop_name}: {val_loss:.4f}")
    print("\nClass Indices:", train_data.class_indices)
    
    return val_acc

# ✅ Train models for all crops
results = {}
for crop in CROPS:
    try:
        acc = train_crop_model(crop)
        results[crop] = acc
    except Exception as e:
        print(f"❌ Error processing {crop}: {str(e)}")
        results[crop] = None

# ✅ Print summary of all results
print("\n\n=== FINAL RESULTS ===")
for crop, acc in results.items():
    if acc is not None:
        print(f"{crop}: {acc:.4f} validation accuracy")
    else:
        print(f"{crop}: Failed to train model")

print("\nTraining completed for all crops!")

Available directories in dataset root:
['Tomato', 'Wheat', 'Sugarcane', 'Corn', 'Cotton', 'Grape', 'Rice', 'Potato']

🚀 Starting training for Corn crop

Preprocessing training data...


Processing Corn_Cercospora_leaf_spot Gray_leaf_spot: 100%|██████████| 2216/2216 [00:13<00:00, 164.93it/s]
Processing Corn_(maize)healthy: 100%|██████████| 1859/1859 [00:09<00:00, 186.17it/s]


Preprocessing validation data...


Processing Corn_Cercospora_leaf_spot Gray_leaf_spot: 100%|██████████| 430/430 [00:02<00:00, 165.36it/s]
Processing Corn_(maize)healthy: 100%|██████████| 465/465 [00:02<00:00, 177.65it/s]


Number of Corn disease classes: 4
Found 9196 images belonging to 4 classes.
Found 1867 images belonging to 4 classes.

✅ Phase 1: Training classifier head for Corn diseases...
Epoch 1/8
288/288 ━━━━━━━━━━━━━━━━━━━━ 233s 653ms/step - accuracy: 0.2971 - loss: 1.4103 - val_accuracy: 0.2651 - val_loss: 1.4321 - learning_rate: 0.0010
Epoch 2/8
288/288 ━━━━━━━━━━━━━━━━━━━━ 159s 553ms/step - accuracy: 0.3553 - loss: 1.3671 - val_accuracy: 0.3637 - val_loss: 1.3913 - learning_rate: 0.0010
Epoch 3/8
288/288 ━━━━━━━━━━━━━━━━━━━━ 155s 538ms/step - accuracy: 0.3584 - loss: 1.3631 - val_accuracy: 0.2817 - val_loss: 1.3780 - learning_rate: 0.0010
Epoch 4/8
288/288 ━━━━━━━━━━━━━━━━━━━━ 157s 546ms/step - accuracy: 0.3477 - loss: 1.3629 - val_accuracy: 0.3235 - val_loss: 1.3669 - learning_rate: 0.0010
Epoch 5/8
288/288 ━━━━━━━━━━━━━━━━━━━━ 155s 537ms/step - accuracy: 0.3645 - loss: 1.3577 - val_accuracy: 0.3091 - val_loss: 1.3782 - learning_rate: 0.0010
Epoch 6/8
288/288 ━━━━━━━━━━━━━━━━━━━━ 0s 519ms/s

...

Processing Rice Healthy: 100%|██████████| 1059/1059 [00:16<00:00, 65.72it/s]


Preprocessing validation data...


Processing Rice Healthy: 100%|██████████| 352/352 [00:05<00:00, 63.31it/s]


Number of Rice disease classes: 9
Found 7610 images belonging to 9 classes.
Found 1611 images belonging to 9 classes.

✅ Phase 1: Training classifier head for Rice diseases...
Epoch 1/8
238/238 ━━━━━━━━━━━━━━━━━━━━ 206s 675ms/step - accuracy: 0.2743 - loss: 2.0170 - val_accuracy: 0.1949 - val_loss: 2.1435 - learning_rate: 0.0010
Epoch 2/8
238/238 ━━━━━━━━━━━━━━━━━━━━ 129s 541ms/step - accuracy: 0.3107 - loss: 1.9059 - val_accuracy: 0.1906 - val_loss: 2.1444 - learning_rate: 0.0010
Epoch 3/8
238/238 ━━━━━━━━━━━━━━━━━━━━ 129s 542ms/step - accuracy: 0.3127 - loss: 1.9016 - val_accuracy: 0.1974 - val_loss: 2.0929 - learning_rate: 0.0010
Epoch 4/8
238/238 ━━━━━━━━━━━━━━━━━━━━ 129s 543ms/step - accuracy: 0.3203 - loss: 1.8857 - val_accuracy: 0.2483 - val_loss: 2.0667 - learning_rate: 0.0010
Epoch 5/8
238/238 ━━━━━━━━━━━━━━━━━━━━ 127s 535ms/step - accuracy: 0.3192 - loss: 1.8750 - val_accuracy: 0.1943 - val_loss: 2.1004 - learning_rate: 0.0010
Epoch 6/8
238/238 ━━━━━━━━━━━━━━━━━━━━ 0s 523ms/s

E0000 00:00:1751592669.485199     115 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1751592669.629211     115 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1751592670.141236     115 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1751592670.288916     115 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
